In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent
sys.path.append(str(BASE_DIR))

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import pandas as pd

In [ ]:
DIR = BASE_DIR/'scripts_res'
DIR

### ASBO

In [5]:
re_run = True
re_run_save = True

In [ ]:
sol = 'ASBO'
case_name = 'Case3'
runs_number = 3

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run == True or re_run_save == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=100, n_iter=200,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        x_3_list.extend([optimizer.max['params']['x3']])
        x_4_list.extend([optimizer.max['params']['x4']])
        x_5_list.extend([optimizer.max['params']['x5']])                        
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        con_4_list.extend([optimizer.max['constraint'][3]])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


In [ ]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### trust-constr

In [8]:
re_run = True
re_run_save = True

In [ ]:
sol = 'trust-constr'
list_realvalue = []
case_name = 'Case3'
runs_number = 3

def func(x):
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1,c2,c3,c4]
    list_realvalue.extend([[obj]+cons])
    return obj,c1,c2,c3,c4

initial_x1 = []
initial_x2 = []
initial_x3 = []
initial_x4 = []
initial_x5 = []

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []

nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
    x_3_list.append(result.x[2])
    x_4_list.append(result.x[3])
    x_5_list.append(result.x[4])        
    con_1_list.append(result.constr[0][0])
    con_2_list.append(result.constr[0][1])
    con_3_list.append(result.constr[0][2])    
    con_4_list.append(result.constr[0][3])    
    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    initial_x3.append(x[0][2])  
    initial_x4.append(x[0][3])  
    initial_x5.append(x[0][4])      
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({ 
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'initial_x3': initial_x3,
        'initial_x4': initial_x4,  
        'initial_x5': initial_x5,                 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run == True or re_run_save == True:
    x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### slsqp

In [11]:
re_run = True
re_run_save = True

In [ ]:
sol = 'slsqp'
case_name = 'Case3'
runs_number = 3

initial_x1 = []
initial_x2 = []
initial_x3 = []
initial_x4 = []
initial_x5 = []

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1,c2,c3,c4]
    list_realvalue.extend([[obj]+cons])
    return obj,c1,c2,c3,c4

initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
    x_3_list.append(result.x[2])
    x_4_list.append(result.x[3])
    x_5_list.append(result.x[4])        

    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    initial_x3.append(x[0][2])  
    initial_x4.append(x[0][3])  
    initial_x5.append(x[0][4])      
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({ 
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'initial_x3': initial_x3,
        'initial_x4': initial_x4,  
        'initial_x5': initial_x5,                 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run == True or re_run_save == True:
    x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### CBO

In [14]:
re_run = True
re_run_save = True

In [ ]:
sol = 'CBO'
case_name = 'Case3'
runs_number = 3

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []

def func_obj(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run == True or re_run_save == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=100, n_iter=200,acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        x_3_list.extend([optimizer.max['params']['x3']])
        x_4_list.extend([optimizer.max['params']['x4']])
        x_5_list.extend([optimizer.max['params']['x5']])                        
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        con_4_list.extend([optimizer.max['constraint'][3]])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,        
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


In [ ]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### TuRBO

In [ ]:
re_run = True
re_run_save = True

In [ ]:
sol = 'TuRBO'
case_name = 'Case3'

temp_y = []
temp_x = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
iterations = []
max_values = []

runs_number = 3
itera = 0
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    itera=itera+1

    return -obj,c1,c2, c3,c4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 1
batch_size = 4
n_init = 100 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1.0, -1, -2.0, 0,], dtype=torch.float64)
ub =  torch.tensor([2.0, 4, 3.0, 1.5, ], dtype=torch.float64)
bounds = torch.tensor([[-5, -20,  -10, -12, -15], [12, 22, 10, 20, 18]])

def objective(x):
    x_numpy = x.numpy()
    res = func(x_numpy)
    target = res[0]
    c1 = res[1]
    c2 = res[2]
    c3 = res[3]
    c4 = res[4]
    return target,c1,c2,c3,c4
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []
            C2_tensor_list = []
            C3_tensor_list = []
            C4_tensor_list = []
            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1, C2,C3,C4 = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)
                C2_tensor_list.append(C2)
                C3_tensor_list.append(C3)
                C4_tensor_list.append(C4)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)
            C2 = torch.stack(C2_tensor_list, dim=0)
            C3 = torch.stack(C3_tensor_list, dim=0)
            C4 = torch.stack(C4_tensor_list, dim=0)
            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)
                c2_model = tr_bo.get_fitted_model(train_X, C2,dim)
                c3_model = tr_bo.get_fitted_model(train_X, C3,dim)
                c4_model = tr_bo.get_fitted_model(train_X, C4,dim)
                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model, c2_model,c3_model,c4_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                C2_next_tensor_list = []
                C3_next_tensor_list = []
                C4_next_tensor_list = []
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next, C2_next, C3_next, C4_next,= combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)
                    C2_next_tensor_list.append(C2_next)
                    C3_next_tensor_list.append(C3_next)
                    C4_next_tensor_list.append(C4_next)
                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C2_next = torch.stack(C2_next_tensor_list, dim=0)
                C3_next = torch.stack(C3_next_tensor_list, dim=0)
                C4_next = torch.stack(C4_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next, C2_next,C3_next, C4_next,], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                C2 = torch.cat((C2, C2_next), dim=0)
                C3 = torch.cat((C3, C3_next), dim=0)
                C4 = torch.cat((C4, C4_next), dim=0)
                C = torch.cat([C1, C2, C3, C4], dim=-1)
                bool_tensor = (C >= lb.repeat(len(C), 1)) & (C <= ub.repeat(len(C), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                Valid_X = train_X[bool_tensor]
                if len(Valid_Y)==0:
                    break
                else:
                    max_index = torch.argmax(Valid_Y)  
                    max_y = Valid_Y[max_index]         
                    corresponding_x = Valid_X[max_index]
                    temp_y.append(max_y)
                    temp_x.append(corresponding_x)
                    # list_realvalue.append(max(Valid_Y[0]))   # Wrong version
                if len(temp_y) == 50:
                    max_value = max(temp_y).item()
                    temp_y_np = np.array([t.item() for t in temp_y])
                    max_index = np.argmax(temp_y_np)
                    # max_index = np.argmax(temp_y)
                    c_x = temp_x[max_index]
                    print("obj= ",max_value, ' x = ', c_x)
                    max_values.append(max_value)
                    x_1_list.append(c_x[0].item())
                    x_2_list.append(c_x[1].item())
                    x_3_list.append(c_x[2].item())
                    x_4_list.append(c_x[3].item())
                    x_5_list.append(c_x[4].item())                                        
                    if len(iterations)==0:
                        iterations.append(itera)
                    else:
                        iterations.append(itera-sum(iterations))
                    temp_y.clear()
                    temp_x.clear()

                    df = pd.DataFrame({
                    'obj': max_values,
                    'x_1': x_1_list,
                    'x_2': x_2_list,
                    'x_3': x_3_list,
                    'x_4': x_4_list,
                    'x_5': x_5_list,                       
                    'iterations': iterations
                    })
                    if re_run_save == True:
                        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                    break

        except Exception as e:
            print(e)
            if len(temp_y)==0:
                print("Early stop and no feasible solutions")
                max_values.append(None)
                x_1_list.append(None)
                x_2_list.append(None)
                x_3_list.append(None)
                x_4_list.append(None)
                x_5_list.append(None)                
            else:
                max_value = max(temp_y).item()
                temp_y_np = np.array([t.item() for t in temp_y])
                max_index = np.argmax(temp_y_np)
                # max_index = np.argmax(temp_y)
                c_x = temp_x[max_index]
                print("Early stop and feasible")
                print("obj= ",max_value, ' x = ', c_x)
                
                max_values.append(max_value)
                x_1_list.append(c_x[0].item())
                x_2_list.append(c_x[1].item())
                x_3_list.append(c_x[2].item())
                x_4_list.append(c_x[3].item())
                x_5_list.append(c_x[4].item())                   
            if len(iterations)==0:
                iterations.append(itera)
            else:
                iterations.append(itera-sum(iterations))
            temp_y.clear()
            temp_x.clear()

            df = pd.DataFrame({
            'obj': max_values,
            'x_1': x_1_list,
            'x_2': x_2_list,
            'x_3': x_3_list,
            'x_4': x_4_list,
            'x_5': x_5_list,               
            'iterations': iterations
            })
            if re_run_save == True:
                df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

run_multiple_calculations(num_runs=runs_number)


In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### PSO

In [14]:
re_run = True
re_run_save = True

In [ ]:
sol = 'PSO'
case_name = 'Case3'

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
iterations=[]

runs_number = 3
itera = 0
from pyswarm import pso

def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1+1,2-c1,c2+1,4-c2,c3+2,3-c3,c4,1.5-c4]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

if re_run == True or re_run_save == True:
    for num in range(runs_number):
        xopt, fopt = pso(lambda x:func(x)[0], lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18],f_ieqcons=lambda x:func(x)[1:2], swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=200, minstep=1e-8,minfunc=1e-8, debug=False)
        print( 'fopt =', fopt, 'xopt = ', xopt)
        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(fopt)
        x_1_list.append(xopt[0])
        x_2_list.append(xopt[1])
        x_3_list.append(xopt[2])
        x_4_list.append(xopt[3])
        x_5_list.append(xopt[4])

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,    
        'iterations': iterations
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


In [ ]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### GA

In [ ]:
re_run = True
re_run_save = True

In [ ]:
sol = 'GA'
case_name = 'Case3'

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
iterations=[]

runs_number = 3
itera = 0
from sko.GA import GA
from sko.operators import crossover

def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    cx1 = -c1-1
    cx11=c1-2
    cx2=-1-c2
    cx22=c2-4
    cx3=-2-c3
    cx33=c3-3
    cx4=-c4
    cx44=c4-1.5
    itera=itera+1
    return obj,cx1,cx2,cx3,cx4,cx11,cx22,cx33,cx44

if re_run == True or re_run_save == True:
    for num in range(runs_number):
        ga = GA(func=lambda x:func(x)[0], n_dim=5, size_pop=100, max_iter=200,lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[i] for i in range(1,9)])
        ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
        best_x, best_y = ga.run()
        print( 'obj =', best_y, 'x = ', best_x)

        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        
        obj_list.append(best_y[0])
        x_1_list.append(best_x[0])
        x_2_list.append(best_x[1])
        x_3_list.append(best_x[2])
        x_4_list.append(best_x[3])
        x_5_list.append(best_x[4])


        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list, 
        'iterations': iterations
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

In [ ]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()